In [1]:
ls

__pycache__/                       df_predict.csv
blocks.csv                         get_predicted.py*
chart.csv                          get_predicted_shebang_namemain.py
dashboard.py                       old/
deploy_clean.py                    rf_test_model.pkl
dev.ipynb                          transactions.csv
dev2.ipynb


In [2]:
import streamlit as st
from streamlit_autorefresh import st_autorefresh
import pandas as pd
import numpy as np
from dashboard import (
    clean_dates,
    clean_predicted,
    surge_chart,
    surge_index,
    gas_hist,
)
from datetime import datetime
import time

2021-12-03 21:58:32.828 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


In [3]:
TZ = "US/Eastern"
files = {
    "blocks" : 'blocks.csv',
    "transactions" : 'transactions.csv',
    "predicted" : 'chart.csv'
}


In [4]:
data = {k : pd.read_csv(v) for k, v in files.items()}
data = clean_dates(data, tz=TZ)


In [5]:
type(data)

dict

In [6]:
data.keys()

dict_keys(['blocks', 'transactions', 'predicted'])

In [7]:
#chart_df = surge_index(data)
pred = data['predicted']

In [8]:
pred.columns

Index(['receipt_effective_gas_price_mean', 'predicted'], dtype='object')

In [9]:
pred.head()

,receipt_effective_gas_price_mean,predicted
block_timestamp,,
2021-11-30 10:49:43-05:00,1.197941e+11,9.119814e+10
2021-11-30 10:50:01-05:00,1.218046e+11,9.119814e+10
2021-11-30 10:50:03-05:00,1.247563e+11,9.119814e+10
2021-11-30 10:50:11-05:00,1.187208e+11,9.327116e+10
2021-11-30 10:50:16-05:00,1.169524e+11,9.211373e+10


In [10]:
this = clean_predicted(data['predicted'])

# Tracy

In [11]:
surge_cht = surge_chart(clean_predicted(data["predicted"]))

In [12]:
st.plotly_chart(surge_cht)


2021-12-03 21:58:35.919 
  command:

    streamlit run /Users/anthonybaldor/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [13]:
rolling_lookback='24h'
quantile=0.25
halflife=15

normalized_data = this / this.rolling(rolling_lookback).quantile(quantile)
idx = (np.log(normalized_data).clip(lower=0) + 1)
smoothed_index = idx.ewm(halflife=halflife).mean()

In [14]:
smoothed_index[~this.isna()]

,receipt_effective_gas_price_mean,predicted
block_timestamp,,
2021-11-30 10:49:43-05:00,1.0,NaN
2021-11-30 10:49:44-05:00,1.0,NaN
2021-11-30 10:49:45-05:00,1.0,NaN
2021-11-30 10:49:46-05:00,1.0,NaN
2021-11-30 10:49:47-05:00,1.0,NaN
...,...,...
2021-11-30 11:13:23-05:00,NaN,1.057081
2021-11-30 11:13:24-05:00,NaN,1.054503
2021-11-30 11:13:25-05:00,NaN,1.052042


# Anthony

In [19]:
thresh_df = surge_index(data['predicted'])

In [20]:
thresh_df

,receipt_effective_gas_price_mean,predicted
block_timestamp,,
2021-11-30 10:49:43-05:00,1.000000,1.000000
2021-11-30 10:50:01-05:00,1.006372,1.000000
2021-11-30 10:50:03-05:00,1.015393,1.000000
2021-11-30 10:50:11-05:00,1.011274,1.006014
2021-11-30 10:50:16-05:00,1.008806,1.006885
...,...,...
2021-11-30 11:11:32-05:00,1.259593,1.136082
2021-11-30 11:12:13-05:00,1.262558,1.133612
2021-11-30 11:12:18-05:00,1.265759,1.129520


# more tests

In [17]:
preds_df = pred.copy().sort_index().resample('s').ffill()


In [18]:
preds_df.head()

,receipt_effective_gas_price_mean,predicted
block_timestamp,,
2021-11-30 10:49:43-05:00,1.197941e+11,9.119814e+10
2021-11-30 10:49:44-05:00,1.197941e+11,9.119814e+10
2021-11-30 10:49:45-05:00,1.197941e+11,9.119814e+10
2021-11-30 10:49:46-05:00,1.197941e+11,9.119814e+10
2021-11-30 10:49:47-05:00,1.197941e+11,9.119814e+10
